# load pre-generated  node text and similar node examples

In [27]:
import json
import random

filename = '../dataset/cora_prompt.json'
with open(filename, 'r') as f:
    cora = json.load(f)



print one sample 

In [28]:
for key in cora["1"]:
    print(key, ":")
    print(cora["1"][key])
    print("\n")

id :
1


lable :
Rule_Learning


sample_text :
{"id": 1, "title": " Applications of machine learning: a medical follow up study  ", "category": "Masked", "keyword": "N/A", "author": " Du Junping K. L. Rasmussen J. Aagaard Brian H. Mayoh Tom Srensen ", "affiliation": " Computer Science Department, Aarhus University,  Department of Obstetrics and Gynaecology, Aarhus University Hospital,  ", "abstract": " This paper describes preliminary work that aims to apply some learning strategies to a medical follow-up study. An investigation of the application of three machine learning algorithms-1R, FOIL and InductH to identify risk factors that govern the colposuspension cure rate has been made. The goal of this study is to induce a generalised description or explanation of the classification attribute, colposuspension cure rate (completely cured, improved, unchanged and worse) from the 767 examples in the questionnaires. We looked for a set of rules that described which risk factors result in di

# Sample around 20% of nodes from the full dataset 

In [29]:
sampled_keys = random.sample(list(cora.keys()), 550)
sampled_cora = {key: cora[key] for key in sampled_keys}


# Inference
insert your openai.api_key
or you can revise the get_completion funtion so that you can use other LLMs

In [34]:
import openai
#model="gpt-3.5-turbo-16k" 
#model="gpt-4" 
model="gpt-4-1106-preview"
openai.organization = ""
openai.api_key = ""
s
def get_completion(prompt, model=model):    
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.1, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [35]:
def get_inductive_prompt(similar_examples):
    
    prompt = similar_examples
    prompt += "\n"
    prompt += f"""
        You are a machine learning expert. For the given example and your own knowledges. 
        Can you find some reasons that why each paper falls into its category. Only briefly list the reasons
        """
    return prompt


def get_deductive_prompt(sample_text, similar_examples, reasons):
    prompt = "Examples: \n"
    prompt += similar_examples
    prompt += "Reasons: \n"
    prompt += reasons
    prompt += "\n"
    prompt += "Target Paper: \n"
    prompt += sample_text + "\n"
    prompt += """
    the "Target Paper" should be in one of which category blow?
    1: 'Case_Based',
    2:'Genetic_Algorithms',
    3.'Neural_Networks',
    4.'Probabilistic_Methods',
    5.'Reinforcement_Learning',
    6.'Rule_Learning',
    7.'Theory'
        
        
    Based on the reasons and examples, think step by step and print reasoning process, 
    then choose the category name.
    output all prediction and reasoning in json format:
    {
        reasoning:, 
        prediction: 
    }
    """
    prompt += '\n'
    return prompt

In [36]:
cnt = 0
tp = 0
verbose = False
data = sampled_cora

for idx in data:
    print("reasoning on sample:" , idx)
    sample = data[idx]
    similar_examples = sample["similar_examples"]
    
    inductive_prompt = get_inductive_prompt(similar_examples)
    reasons = get_completion(inductive_prompt)
    sample_text = sample["sample_text"]
    deductive_prompt = get_deductive_prompt(sample_text, similar_examples, reasons)
    res = get_completion(deductive_prompt)

    
    if model == "gpt-4-1106-preview":
        start = res.find("json") + len("json")
        end = res.find("```", start)
        json_string = res[start:end].strip()
        res_dict = json.loads(json_string)
    elif model == "gpt-4":
        start = res.find("{") - 1
        end = res.find("}", start) + 1
        json_string = res[start:end].strip()
        res_dict = json.loads(json_string)
    else: 
        json_string = json.loads(res)
    
    if verbose:
        #print("similar_examples: \n", similar_examples, "\n")
        #print("reasons: \n", reasons, "\n")
        print("deductive_prompt: \n",deductive_prompt, "\n")
        print("result: \n", res, "\n")
        print("json_result: \n", res_dict, "\n")
        print("lable:", sample["lable"],"\n")
        print("===========================================")
    
    if res_dict["prediction"] == sample["lable"]:
        tp += 1
    cnt += 1
    

In [33]:
tp/cnt

1.0